In [1]:
#import libs to work
import pandas as pd


In [2]:
#function to return the dataset in pandas form.
def load_diabetes_data():
    url = ('https://raw.githubusercontent.com/abdelaziztestas/spark_book/main/diabetes.csv')
    return pd.read_csv(url)

In [3]:
pandas_df = load_diabetes_data()
pandas_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [17]:
#now in spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean

# Crear la sesión Spark
spark = SparkSession.builder \
    .appName("diabetes_data") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.cores", "4") \
    .master("local[*]") \
    .getOrCreate()

spark_df = spark.read.csv('C:/Users/hedin/Documents/pyspark-the correct way/notebooks/diabetes.csv',header=True, inferSchema=True)

In [18]:
spark_df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [6]:
pandas_df.drop('Outcome', axis=1).describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000


In [19]:
spark_df.drop('Outcome').summary().show()

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------------+------------------+
|summary|       Pregnancies|          Glucose|     BloodPressure|     SkinThickness|           Insulin|               BMI|DiabetesPedigreeFunction|               Age|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------------+------------------+
|  count|               768|              768|               768|               768|               768|               768|                     768|               768|
|   mean|3.8450520833333335|     120.89453125|       69.10546875|20.536458333333332| 79.79947916666667|31.992578124999977|      0.4718763020833327|33.240885416666664|
| stddev|  3.36957806269887|31.97261819513622|19.355807170644777|15.952217567727642|115.24400235133803| 7.884160320375441|       0.331328595012775|11.760231540678689

In [15]:
pandas_df["Outcome"].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [20]:
spark_df.groupBy("Outcome").count().show()

+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  268|
|      0|  500|
+-------+-----+



In [21]:
pandas_df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [22]:
from pyspark.sql.functions import sum, col
spark_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in spark_df.columns]).show()

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|          0|      0|            0|            0|      0|  0|                       0|  0|      0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [23]:
print((pandas_df == 0).sum())

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64


In [24]:
exprs = [sum((col(c) == 0).cast("int")).alias(c) for c in
spark_df.columns]

In [25]:
spark_df.select(exprs).show()

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|        111|      5|           35|          227|    374| 11|                       0|  0|    500|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [26]:
pandas_df = pandas_df.loc[(pandas_df['Glucose'] != 0)&
(pandas_df['BloodPressure'] != 0)& (pandas_df['BMI'] != 0), ['Pregnancies',
'Glucose', 'BloodPressure', 'BMI', 'DiabetesPedigreeFunction', 'Age',
'Outcome']]

In [27]:
spark_df = spark_df.filter((col('Glucose') != 0)&
(col('BloodPressure') != 0)& (col('BMI') != 0)).select(['Pregnancies',
'Glucose', 'BloodPressure', 'BMI', 'DiabetesPedigreeFunction', 'Age',
'Outcome'])

In [28]:
print(pandas_df.shape)

(724, 7)


In [29]:
print((spark_df.count(), len(spark_df.columns)))

(724, 7)
